In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) #makes the display bars longer and spread out 100% across the screen width

In [2]:
from vivarium import Artifact
import gbd_mapping as gbd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

# These are runs from 2020 summer -- run 002.2test for India BIRTHWEIGHT CHECKS 
- these are the most recent runs with updated fixes

Test run completed, find results here:


/share/costeffectiveness/results/vivarium_gates_bep/summer_002.2test/india/2020_10_11_16_39_49/


This is one country (India) using the configuration we have termed "Run2". 

It is up to date with the exception of the latest change to maternal bw shift. 

-It does contain the change to report z-scores at 2 timepoints (29 and 366 days). 
-bug fix from betr distribution for z-scores (baseline)

-It is useful for checking the following PRs:
1) Add new BEP birthweight effect sizes #102 (https://github.com/ihmeuw/vivarium_gates_bep/pull/102/files/e594e6dc10d2574ba7c81e09affbb54d3840b923)
2) Use the correct column when assigning values to stunting z-score #106 (https://github.com/ihmeuw/vivarium_gates_bep/pull/106)
3) Record wasting and stunting z-scores at 29 days and 366 days #108 (https://github.com/ihmeuw/vivarium_gates_bep/pull/108)


file location: /share/costeffectiveness/results/vivarium_gates_bep/summer_run1/

In [6]:
%cd /share/costeffectiveness/results/vivarium_gates_bep/summer_002.2test/india/2020_10_11_16_39_49/count_data/ 
%ls

!whoami
!date

/ihme/costeffectiveness/results/vivarium_gates_bep/summer_002.2test/india/2020_10_11_16_39_49/count_data
birth_weight.csv    deaths.csv           population.csv         ylds.csv
birth_weight.hdf    deaths.hdf           population.hdf         ylds.hdf
cgf_categories.csv  gestational_age.csv  state_person_time.csv  ylls.csv
cgf_categories.hdf  gestational_age.hdf  state_person_time.hdf  ylls.hdf
cgf_z_scores.csv    person_time.csv      transition_count.csv
cgf_z_scores.hdf    person_time.hdf      transition_count.hdf
nicoly
Tue Oct 13 21:16:33 PDT 2020


In [11]:
#scenarios in each of the runs

#run1 (baseline, mmn, bep_hd_universal_i & targeted_i)
#run2 (bep_ce_universal & targeted)
#run3 (bep_ce_universal & targeted)
#run4 (baseline, mmn, bep_hd_universal_iii & targeted_iii)

output_dirs001 = []

output_dirs002 = ['/share/costeffectiveness/results/vivarium_gates_bep/summer_002.2test/india/2020_10_11_16_39_49/count_data/']

output_dirs004 = []

#location labels
locations = ['India','Pakistan','Mali','Tanzania']

#use in sort_values to make custum sort list. I sorted by region. 
sorter = ['India','mali','Mali','Tanzania']
sorterIndex = dict(zip(sorter,range(len(sorter))))


# EXPLORING: INDIA by scenario and treatment groups

In [8]:
bw_raw_india = pd.read_hdf('/share/costeffectiveness/results/vivarium_gates_bep/summer_002.2test/india/2020_10_11_16_39_49/count_data/birth_weight.hdf')
bw_raw_india.head()

,risk,treatment_group,mother_status,measure,input_draw,scenario,value
0,birth_weight,bep,malnourished,mean,3,bep_ce_scale_up,2904.813493
1,birth_weight,bep,malnourished,mean,3,bep_ce_targeted_scale_up,2904.813493
2,birth_weight,bep,malnourished,mean,21,bep_ce_scale_up,2957.068371
3,birth_weight,bep,malnourished,mean,21,bep_ce_targeted_scale_up,2957.068371
4,birth_weight,bep,malnourished,mean,29,bep_ce_scale_up,2900.532754


In [9]:
pop_raw_india = pd.read_hdf(/share/costeffectiveness/results/vivarium_gates_bep/summer_002.2test/india/2020_10_11_16_39_49/count_data/population.hdf')
pop_raw_india.head()

SyntaxError: invalid syntax (<ipython-input-9-40241bfc73f4>, line 1)

In [12]:
#RUNs appended 
# left out run 3 because ii correlation coefficient very similar to iii


master_bw= pd.DataFrame()

for i in list(range(len(output_dirs002))):
    
    #birthweight
    #bw_001 = pd.read_hdf(output_dirs001[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'birthweight'})
    #bw_001['scenario'] = bw_001['scenario'].replace('bep_hd_scale_up', 'bep_hd_scale_up_i').replace('bep_hd_targeted_scale_up','bep_hd_targeted_scale_up_i')
    
    bw_002 = pd.read_hdf(output_dirs002[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'birthweight'})
  
    #bw_004 = pd.read_hdf(output_dirs004[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status']).rename(columns={'value':'birthweight'})
    #bw_004['scenario'] = bw_004['scenario'].replace('bep_hd_scale_up', 'bep_hd_scale_up_iii').replace('bep_hd_targeted_scale_up','bep_hd_targeted_scale_up_iii')
    
    #bw = bw_001.append([bw_002, bw_004])
    bw = bw_002
    
    #population
    #pop_001 = pd.read_hdf(output_dirs001[i] + 'population.hdf').reset_index()
    #pop_001 = pop_001.where(pop_001['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])
    #pop_001['scenario'] = pop_001['scenario'].replace('bep_hd_scale_up', 'bep_hd_scale_up_i').replace('bep_hd_targeted_scale_up','bep_hd_targeted_scale_up_i')
    
    pop_002 = pd.read_hdf(output_dirs002[i] + 'population.hdf').reset_index()
    pop_002 = pop_002.where(pop_002['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])
    
    #pop_004 = pd.read_hdf(output_dirs001[i] + 'population.hdf').reset_index()
    #pop_004 = pop_004.where(pop_004['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])
    #pop_004['scenario'] = pop_004['scenario'].replace('bep_hd_scale_up', 'bep_hd_scale_up_iii').replace('bep_hd_targeted_scale_up','bep_hd_targeted_scale_up_iii')
    
    #pop = pop_001.append([pop_002, pop_004])   
    pop = pop_002
    
    data = bw.merge(pop, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    data = data.groupby(['risk','measure','scenario','treatment_group','mother_status']).mean().drop(columns=['input_draw','index']).reset_index() #average over input draws
    data['pop_x_bw'] = data['birthweight'] * data['total_population']
    data['location'] = locations[i]
    master_bw = master_bw.append(data)
    
master_bw.head()


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
0,birth_weight,mean,bep_ce_scale_up,bep,malnourished,2911.467743,13444.32,3.914270e+07,India
1,birth_weight,mean,bep_ce_scale_up,bep,normal,3012.287399,65971.26,1.987244e+08,India
2,birth_weight,mean,bep_ce_scale_up,ifa,malnourished,0.000000,0.00,0.000000e+00,India
3,birth_weight,mean,bep_ce_scale_up,ifa,normal,0.000000,0.00,0.000000e+00,India
4,birth_weight,mean,bep_ce_scale_up,mmn,malnourished,0.000000,0.00,0.000000e+00,India


In [13]:
master_bw['scenario'].unique()

array(['bep_ce_scale_up', 'bep_ce_targeted_scale_up'], dtype=object)

In [14]:
master_bw['treatment_group'].unique()

array(['bep', 'ifa', 'mmn', 'none'], dtype=object)

In [15]:
master_bw['measure'].unique()

array(['mean', 'proportion_below_2500g', 'sd'], dtype=object)

# ****** -- CHECK birthweight shifts by scenario -- ******


***************************************************************************************************


## *BASELINE

In [11]:
#to get the dataframe we want from run1 (BASELINE):
#INDIA

india_bw_baseline = master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]
india_bwsd_baseline = master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['sd']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]

print(india_bw_baseline['pop_x_bw'].sum()/100_000)
print(india_bwsd_baseline['pop_x_bw'].sum()/100_000)
india_bw_baseline

2886.3580144210277
691.6361147997239


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
2,birth_weight,mean,baseline,ifa,malnourished,2801.630311,5770.46,1.616670e+07,India
3,birth_weight,mean,baseline,ifa,normal,2951.342171,28262.46,8.341219e+07,India
6,birth_weight,mean,baseline,none,malnourished,2740.603228,11166.04,3.060169e+07,India
7,birth_weight,mean,baseline,none,normal,2891.463930,54801.04,1.584552e+08,India


In [12]:
#pakistan

pakistan_bw_baseline = master_bw[master_bw['location'].isin(['Pakistan']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]
pakistan_bwsd_baseline = master_bw[master_bw['location'].isin(['Pakistan']) & master_bw['measure'].isin(['sd']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]

print(pakistan_bw_baseline['pop_x_bw'].sum()/100_000)
print(pakistan_bwsd_baseline['pop_x_bw'].sum()/100_000)
pakistan_bw_baseline 

2839.522665772495
698.0559711192248


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
2,birth_weight,mean,baseline,ifa,malnourished,2749.523346,2668.90,7.338203e+06,Pakistan
3,birth_weight,mean,baseline,ifa,normal,2900.198665,22041.16,6.392374e+07,Pakistan
6,birth_weight,mean,baseline,none,malnourished,2689.173309,8109.32,2.180737e+07,Pakistan
7,birth_weight,mean,baseline,none,normal,2841.339571,67180.62,1.908830e+08,Pakistan


In [13]:
#MALI

mali_bw_baseline = master_bw[master_bw['location'].isin(['Mali']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]
mali_bwsd_baseline = master_bw[master_bw['location'].isin(['Mali']) & master_bw['measure'].isin(['sd']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]
mali_bw_baseline 

print(mali_bw_baseline['pop_x_bw'].sum()/100_000)
print(mali_bwsd_baseline['pop_x_bw'].sum()/100_000)
mali_bw_baseline 

2929.984336992968
660.104598378042


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
2,birth_weight,mean,baseline,ifa,malnourished,2844.261299,2421.32,6.886867e+06,Mali
3,birth_weight,mean,baseline,ifa,normal,2991.457033,20865.44,6.241807e+07,Mali
6,birth_weight,mean,baseline,none,malnourished,2785.095870,7952.18,2.214758e+07,Mali
7,birth_weight,mean,baseline,none,normal,2931.105425,68761.06,2.015459e+08,Mali


In [14]:
#Tanzania

tanzania_bw_baseline = master_bw[master_bw['location'].isin(['Tanzania']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]
tanzania_bwsd_baseline = master_bw[master_bw['location'].isin(['Tanzania']) & master_bw['measure'].isin(['sd']) & master_bw['scenario'].isin(['baseline']) & master_bw['treatment_group'].isin(['ifa','none'])]
tanzania_bw_baseline 

print(tanzania_bw_baseline['pop_x_bw'].sum()/100_000)
print(tanzania_bwsd_baseline['pop_x_bw'].sum()/100_000)
tanzania_bw_baseline 

3274.713703128642
636.1296204147264


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
2,birth_weight,mean,baseline,ifa,malnourished,3192.706552,2014.68,6.432282e+06,Tanzania
3,birth_weight,mean,baseline,ifa,normal,3334.741602,18957.34,6.321783e+07,Tanzania
6,birth_weight,mean,baseline,none,malnourished,3133.742860,7553.38,2.367035e+07,Tanzania
7,birth_weight,mean,baseline,none,normal,3276.001646,71474.60,2.341509e+08,Tanzania


# SCENARIOS 

## MMN SCALE-UP

In [15]:
#to get the dataframe we want from run1-TEST(MMN_SCALE_UP):

bw_mmn = master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['mmn_scale_up']) & master_bw['treatment_group'].isin(['mmn','none'])]
bw_mmn


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
60,birth_weight,mean,mmn_scale_up,mmn,malnourished,2844.616155,13444.32,3.824393e+07,India
61,birth_weight,mean,mmn_scale_up,mmn,normal,2995.833727,65971.26,1.976389e+08,India
62,birth_weight,mean,mmn_scale_up,none,malnourished,2743.978862,3492.18,9.582468e+06,India
63,birth_weight,mean,mmn_scale_up,none,normal,2891.357430,17092.24,4.941978e+07,India


## BEP HD SCALE-UP UNIVERSAL

In [45]:
#to get the dataframe we want from run1-TEST (BEP_HD_SCALE_UP iii):

bw_bep_hd_universal = master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['bep_hd_scale_up_iii']) & master_bw['treatment_group'].isin(['bep','none'])]
bw_bep_hd_universal

,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
32,birth_weight,mean,bep_hd_scale_up_iii,bep,malnourished,2944.582646,13444.32,3.958791e+07,India
33,birth_weight,mean,bep_hd_scale_up_iii,bep,normal,3070.800218,65971.26,2.025846e+08,India
38,birth_weight,mean,bep_hd_scale_up_iii,none,malnourished,2743.978862,3492.18,9.582468e+06,India
39,birth_weight,mean,bep_hd_scale_up_iii,none,normal,2891.357430,17092.24,4.941978e+07,India


## BEP HD TARGETED SCALE UP

In [17]:
#to get the dataframe we want from  (BEP_HD_TARGETED_SCALE_UP iii):

bw_bep_hd_targeted= master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['bep_hd_targeted_scale_up_iii']) & master_bw['treatment_group'].isin(['bep','mmn','none'])]
bw_bep_hd_targeted

,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
48,birth_weight,mean,bep_hd_targeted_scale_up_iii,bep,malnourished,2944.582646,13444.32,3.958791e+07,India
49,birth_weight,mean,bep_hd_targeted_scale_up_iii,bep,normal,0.000000,0.00,0.000000e+00,India
52,birth_weight,mean,bep_hd_targeted_scale_up_iii,mmn,malnourished,0.000000,0.00,0.000000e+00,India
53,birth_weight,mean,bep_hd_targeted_scale_up_iii,mmn,normal,2995.833727,65971.26,1.976389e+08,India
54,birth_weight,mean,bep_hd_targeted_scale_up_iii,none,malnourished,2743.978862,3492.18,9.582468e+06,India
55,birth_weight,mean,bep_hd_targeted_scale_up_iii,none,normal,2891.357430,17092.24,4.941978e+07,India


## BEP CE UNIVERSAL SCALE UP

In [18]:
!whoami
!date

nicoly
Tue Oct 13 21:31:39 PDT 2020


In [16]:
#to get the dataframe we want from run1-TEST (BEP_HD_SCALE_UP iii):

bw_bep_ce_universal = master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['bep_ce_scale_up']) & master_bw['treatment_group'].isin(['bep','none'])]
bw_bep_ce_universal

,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
0,birth_weight,mean,bep_ce_scale_up,bep,malnourished,2911.467743,13444.32,3.914270e+07,India
1,birth_weight,mean,bep_ce_scale_up,bep,normal,3012.287399,65971.26,1.987244e+08,India
6,birth_weight,mean,bep_ce_scale_up,none,malnourished,2743.978862,3492.18,9.582468e+06,India
7,birth_weight,mean,bep_ce_scale_up,none,normal,2891.357430,17092.24,4.941978e+07,India


## BEP CE TARGETED SCALE UP

In [19]:
!whoami
!date

nicoly
Tue Oct 13 21:31:49 PDT 2020


In [17]:
#to get the dataframe we want from  (BEP_HD_TARGETED_SCALE_UP iii):

bw_bep_ce_targeted= master_bw[master_bw['location'].isin(['India']) & master_bw['measure'].isin(['mean']) & master_bw['scenario'].isin(['bep_ce_targeted_scale_up']) & master_bw['treatment_group'].isin(['bep','mmn','none'])]
bw_bep_ce_targeted

,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_bw,location
8,birth_weight,mean,bep_ce_targeted_scale_up,bep,malnourished,2911.467743,13444.32,3.914270e+07,India
9,birth_weight,mean,bep_ce_targeted_scale_up,bep,normal,0.000000,0.00,0.000000e+00,India
12,birth_weight,mean,bep_ce_targeted_scale_up,mmn,malnourished,0.000000,0.00,0.000000e+00,India
13,birth_weight,mean,bep_ce_targeted_scale_up,mmn,normal,2995.833727,65971.26,1.976389e+08,India
14,birth_weight,mean,bep_ce_targeted_scale_up,none,malnourished,2743.978862,3492.18,9.582468e+06,India
15,birth_weight,mean,bep_ce_targeted_scale_up,none,normal,2891.357430,17092.24,4.941978e+07,India


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~




# ======== VALIDATION OF BASELINE WITH GBD ARTIFACT FOR BIRTHWEIGHT ===============

# 1. get population sex distribution from artifact

we collapsed over sex in out outputs.hdf (we didn't ask the engineers to give us sex stratifications
however, the 

In [6]:
# get population birth sex ratio from artifact data

locations = ['india', 'pakistan', 'mali', 'tanzania']

def load_artifact_sr():
    tab = pd.DataFrame()
    for i in locations:
        art = Artifact('/share/costeffectiveness/artifacts/vivarium_gates_bep/' + i + '.hdf', filter_terms=['year_start == 2017', 'age_start == 0'])
        print(i)
        sr = art.load('population.structure').reset_index()
        sr = sr.rename(columns={'value':'pop'})
        sr['total_pop'] = sr['pop'][0] + sr['pop'][1]
        sr['sex_ratio'] = sr['pop'] / sr['total_pop']
        tab=tab.append(sr)
    return tab


In [7]:
sex_ratio = load_artifact_sr()
sex_ratio

india
pakistan
mali
tanzania


,location,sex,age_start,age_end,year_start,year_end,pop,total_pop,sex_ratio
0,India,Female,0.0,0.019178,2017,2018,225403.558880,473130.847643,0.476409
1,India,Male,0.0,0.019178,2017,2018,247727.288763,473130.847643,0.523591
0,Pakistan,Female,0.0,0.019178,2017,2018,53994.159557,112567.291362,0.479661
1,Pakistan,Male,0.0,0.019178,2017,2018,58573.131805,112567.291362,0.520339
0,Mali,Female,0.0,0.019178,2017,2018,8184.729977,16778.605515,0.487808
1,Mali,Male,0.0,0.019178,2017,2018,8593.875539,16778.605515,0.512192
0,Tanzania,Female,0.0,0.019178,2017,2018,18631.222060,37630.501139,0.495110
1,Tanzania,Male,0.0,0.019178,2017,2018,18999.279079,37630.501139,0.504890


# 2. Get LBWSG exposure categories by country FROM GBD (artifact)


from  from Ali's -- LBWSG exposure, rrs, pafs -- noteook

In [22]:
def get_lbwsg_exposure_and_rr_data(location):
    art = pd.HDFStore('/share/costeffectiveness/artifacts/vivarium_gates_bep/' + location + '.hdf')
    keys = art.keys()
    exposure_index = art.get('/risk_factor/low_birth_weight_and_short_gestation/exposure/index')
    exposure_data = art.get('/risk_factor/low_birth_weight_and_short_gestation/exposure/draw_0')
    exposure = exposure_index.merge(exposure_data, right_index=True, left_index=True)
    exposure = (exposure.rename(columns={'draw_0':'exposure'})
                .where(exposure['year_start'] == 2017)
                .where(exposure['age_start'] == 0).dropna()) #these are the 0-7 day years olds
    
    relative_risk_index = art.get('/risk_factor/low_birth_weight_and_short_gestation/relative_risk/index')
    relative_risk_data = art.get('/risk_factor/low_birth_weight_and_short_gestation/relative_risk/draw_0')
    relative_risk = relative_risk_index.merge(relative_risk_data, right_index=True, left_index=True)
    relative_risk = (relative_risk.rename(columns={'draw_0':'relative_risk'})
                .where(relative_risk['year_start'] == 2017)
                .where(relative_risk['age_start'] < 1.92e-02).dropna())
    
    data = exposure.merge(relative_risk, right_on=['location','sex','age_start','age_end',
                                                   'year_start','year_end','parameter'],
                         left_on=['location','sex','age_start','age_end','year_start','year_end','parameter'])
    return data


In [23]:
pwd

'/ihme/costeffectiveness/results/vivarium_gates_bep/summer_run1/india/2020_09_04_17_40_19/count_data'

Defining the lbw tables by country

In [24]:
#total of 228 rows

lbwart_india= get_lbwsg_exposure_and_rr_data('india')
lbwart_pakistan = get_lbwsg_exposure_and_rr_data('pakistan')
lbwart_mali = get_lbwsg_exposure_and_rr_data('mali')
lbwart_tanzania = get_lbwsg_exposure_and_rr_data('tanzania')
#lbw_mali.to_csv('/ihme/homes/nicoly/notebooks/Project BEP/Validation/lbw_mali')
lbwart_mali.head()

,location,sex,age_start,age_end,year_start,year_end,parameter,exposure,affected_entity,affected_measure,relative_risk
0,Mali,Female,0.0,0.019178,2017.0,2018.0,cat10,0.002128,all,excess_mortality_rate,968.702386
1,Mali,Female,0.0,0.019178,2017.0,2018.0,cat106,0.001562,all,excess_mortality_rate,171.751901
2,Mali,Female,0.0,0.019178,2017.0,2018.0,cat11,0.001454,all,excess_mortality_rate,513.870580
3,Mali,Female,0.0,0.019178,2017.0,2018.0,cat116,0.003774,all,excess_mortality_rate,182.396469
4,Mali,Female,0.0,0.019178,2017.0,2018.0,cat117,0.014307,all,excess_mortality_rate,61.431240


In [25]:
lbwart_mali.relative_risk.nunique()
#print(len(relativerisk))

111

In [26]:
lbwart_mali.parameter.nunique()
#print(len(cat))

57

In [27]:
sex = lbwart_mali.sex.unique()
#print(len(cat))
sex

array(['Female', 'Male'], dtype=object)

In [28]:
sex = lbwart_mali.age_start.unique()
#print(len(cat))
sex

array([0.])

## Get continuous birthweight by category using function from Abie's notebook

In [29]:
categories = gbd.risk_factors.low_birth_weight_and_short_gestation.categories

In [30]:
# make dataframe that maps from each category to gestatation_age_start,
# gestatation_age_end, birth_weight_start, and birth weight end

s = pd.Series(categories.to_dict())


In [31]:
# map mean birthweight to cat

def ga_bw_list(cat):
    numbers_only = cat.replace('Birth prevalence - [', '') \
                    .replace(',', '')\
                    .replace(') wks [', ' ')\
                    .replace(') g', '')
    numbers_only = numbers_only.split()
    numbers_only = [int(numbers_only[i]) for i in range(4)]
    
    return numbers_only

i = np.random.randint(len(s))
s.iloc[i], ga_bw_list(s.iloc[i])

('Birth prevalence - [34, 36) wks, [4000, 4500) g', [34, 36, 4000, 4500])

In [32]:
#test equation

l = ga_bw_list(s['cat10'])
(l[2] + l[3]) / 2

750.0

In [33]:
lbwart_mali.head()

,location,sex,age_start,age_end,year_start,year_end,parameter,exposure,affected_entity,affected_measure,relative_risk
0,Mali,Female,0.0,0.019178,2017.0,2018.0,cat10,0.002128,all,excess_mortality_rate,968.702386
1,Mali,Female,0.0,0.019178,2017.0,2018.0,cat106,0.001562,all,excess_mortality_rate,171.751901
2,Mali,Female,0.0,0.019178,2017.0,2018.0,cat11,0.001454,all,excess_mortality_rate,513.870580
3,Mali,Female,0.0,0.019178,2017.0,2018.0,cat116,0.003774,all,excess_mortality_rate,182.396469
4,Mali,Female,0.0,0.019178,2017.0,2018.0,cat117,0.014307,all,excess_mortality_rate,61.431240


# ---------------------------------------------------------------------------------------------------------------------

# OVERALL BIRTHWEIGHT COMPARISONS FOR V & V (artifact and baseline model)

In [34]:
#sex ratio from artficat data

sex_ratio

,location,sex,age_start,age_end,year_start,year_end,pop,total_pop,sex_ratio
0,India,Female,0.0,0.019178,2017,2018,225403.558880,473130.847643,0.476409
1,India,Male,0.0,0.019178,2017,2018,247727.288763,473130.847643,0.523591
0,Pakistan,Female,0.0,0.019178,2017,2018,53994.159557,112567.291362,0.479661
1,Pakistan,Male,0.0,0.019178,2017,2018,58573.131805,112567.291362,0.520339
0,Mali,Female,0.0,0.019178,2017,2018,8184.729977,16778.605515,0.487808
1,Mali,Male,0.0,0.019178,2017,2018,8593.875539,16778.605515,0.512192
0,Tanzania,Female,0.0,0.019178,2017,2018,18631.222060,37630.501139,0.495110
1,Tanzania,Male,0.0,0.019178,2017,2018,18999.279079,37630.501139,0.504890


## INDIA

note: sd from output.hdf are the individual sd , while the measure of spread in the artficat are the standard errors...hence, dont think there is a easy way to compare sds. 

In [39]:
# add mean birthweight to india 

# lbw_india.mean_bw = ga_bw_list(s[lbw_india.parameter])
lbwart_india['mean_bw'] = lbwart_india['parameter'].apply(lambda x: (ga_bw_list(s[x])[2]+ga_bw_list(s[x])[3])/2)
lbwart_india['sex_ratio'] = np.where(lbwart_india['sex']== 'Female', 0.476409, 1-0.476409)
lbwart_india['weighted_bw'] = lbwart_india['exposure'] * lbwart_india['mean_bw'] * lbwart_india['sex_ratio']

print('overall mean birthweight from artifact baseline (india) = ', + lbwart_india['weighted_bw'].sum())
print('overall mean birthweight from model baseline (india) = ', + india_bw_baseline['pop_x_bw'].sum()/100_000, )
print('overall mean sd of birthweight from model baseline (india) = ', + india_bwsd_baseline['pop_x_bw'].sum()/100_000)

lbwart_india.head()

overall mean birthweight from artifact baseline (india) =  2882.5449288053464
overall mean birthweight from model baseline (india) =  2886.3580144210277
overall mean sd of birthweight from model baseline (india) =  691.6361147997239


,location,sex,age_start,age_end,year_start,year_end,parameter,exposure,affected_entity,affected_measure,relative_risk,mean_bw,sex_ratio,weighted_bw
0,India,Female,0.0,0.019178,2017.0,2018.0,cat10,0.000590,all,excess_mortality_rate,968.702386,750.0,0.476409,0.210744
1,India,Female,0.0,0.019178,2017.0,2018.0,cat106,0.003773,all,excess_mortality_rate,171.751901,1250.0,0.476409,2.246593
2,India,Female,0.0,0.019178,2017.0,2018.0,cat11,0.000582,all,excess_mortality_rate,513.870580,750.0,0.476409,0.208009
3,India,Female,0.0,0.019178,2017.0,2018.0,cat116,0.005572,all,excess_mortality_rate,182.396469,1250.0,0.476409,3.318133
4,India,Female,0.0,0.019178,2017.0,2018.0,cat117,0.017522,all,excess_mortality_rate,61.431240,1750.0,0.476409,14.608434


## PAKISTAN

In [40]:
# add mean birthweight to pakistan 

# lbw_pakistan.mean_bw = ga_bw_list(s[lbw_pakistan.parameter])
lbwart_pakistan['mean_bw'] = lbwart_pakistan['parameter'].apply(lambda x: (ga_bw_list(s[x])[2]+ga_bw_list(s[x])[3])/2)
lbwart_pakistan['sex_ratio'] = np.where(lbwart_pakistan['sex']== 'Female', 0.479661, 1-0.479661)
lbwart_pakistan['weighted_bw'] = lbwart_pakistan['exposure'] * lbwart_pakistan['mean_bw'] * lbwart_pakistan['sex_ratio']

print('overall mean birthweight from artifact baseline (pakistan) = ', + lbwart_pakistan['weighted_bw'].sum())
print('overall mean birthweight from model baseline (pakistan) = ', + pakistan_bw_baseline['pop_x_bw'].sum()/100_000, )
print('overall mean sd of birthweight from model baseline (pakistan) = ', + pakistan_bwsd_baseline['pop_x_bw'].sum()/100_000)

lbwart_pakistan.head()

overall mean birthweight from artifact baseline (pakistan) =  2840.9029325769707
overall mean birthweight from model baseline (pakistan) =  2839.522665772495
overall mean sd of birthweight from model baseline (pakistan) =  698.0559711192248


,location,sex,age_start,age_end,year_start,year_end,parameter,exposure,affected_entity,affected_measure,relative_risk,mean_bw,sex_ratio,weighted_bw
0,Pakistan,Female,0.0,0.019178,2017.0,2018.0,cat10,0.000000,all,excess_mortality_rate,968.702386,750.0,0.479661,0.000000
1,Pakistan,Female,0.0,0.019178,2017.0,2018.0,cat106,0.004062,all,excess_mortality_rate,171.751901,1250.0,0.479661,2.435586
2,Pakistan,Female,0.0,0.019178,2017.0,2018.0,cat11,0.000542,all,excess_mortality_rate,513.870580,750.0,0.479661,0.195109
3,Pakistan,Female,0.0,0.019178,2017.0,2018.0,cat116,0.005284,all,excess_mortality_rate,182.396469,1250.0,0.479661,3.168183
4,Pakistan,Female,0.0,0.019178,2017.0,2018.0,cat117,0.019681,all,excess_mortality_rate,61.431240,1750.0,0.479661,16.520460


## MALI

In [41]:
# add mean birthweight to mali 

lbwart_mali['mean_bw'] = lbwart_mali['parameter'].apply(lambda x: (ga_bw_list(s[x])[2]+ga_bw_list(s[x])[3])/2)
lbwart_mali['sex_ratio'] = np.where(lbwart_mali['sex']== 'Female', 0.487808, 1-0.487808)
lbwart_mali['weighted_bw'] = lbwart_mali['exposure'] * lbwart_mali['mean_bw'] * lbwart_mali['sex_ratio']

print('overall mean birthweight from artifact baseline (mali) = ', + lbwart_mali['weighted_bw'].sum())
print('overall mean birthweight from model baseline (mali) = ', + mali_bw_baseline['pop_x_bw'].sum()/100_000, )
print('overall mean sd of birthweight from model baseline (mali) = ', + mali_bwsd_baseline['pop_x_bw'].sum()/100_000)

lbwart_mali.head()

overall mean birthweight from artifact baseline (mali) =  2930.2635119015226
overall mean birthweight from model baseline (mali) =  2929.984336992968
overall mean sd of birthweight from model baseline (mali) =  660.104598378042


,location,sex,age_start,age_end,year_start,year_end,parameter,exposure,affected_entity,affected_measure,relative_risk,mean_bw,sex_ratio,weighted_bw
0,Mali,Female,0.0,0.019178,2017.0,2018.0,cat10,0.002128,all,excess_mortality_rate,968.702386,750.0,0.487808,0.778673
1,Mali,Female,0.0,0.019178,2017.0,2018.0,cat106,0.001562,all,excess_mortality_rate,171.751901,1250.0,0.487808,0.952270
2,Mali,Female,0.0,0.019178,2017.0,2018.0,cat11,0.001454,all,excess_mortality_rate,513.870580,750.0,0.487808,0.531772
3,Mali,Female,0.0,0.019178,2017.0,2018.0,cat116,0.003774,all,excess_mortality_rate,182.396469,1250.0,0.487808,2.301030
4,Mali,Female,0.0,0.019178,2017.0,2018.0,cat117,0.014307,all,excess_mortality_rate,61.431240,1750.0,0.487808,12.213578


## Tanzania

In [42]:
# add mean birthweight to tanzania 

# lbw_tanzania.mean_bw = ga_bw_list(s[lbw_tanzania.parameter])
lbwart_tanzania['mean_bw'] = lbwart_tanzania['parameter'].apply(lambda x: (ga_bw_list(s[x])[2]+ga_bw_list(s[x])[3])/2)
lbwart_tanzania['sex_ratio'] = np.where(lbwart_tanzania['sex']== 'Female', 0.487808, 1-0.487808)
lbwart_tanzania['weighted_bw'] = lbwart_tanzania['exposure'] * lbwart_tanzania['mean_bw'] * lbwart_tanzania['sex_ratio']

print('overall mean birthweight from artifact baseline (tanzania) = ', + lbwart_tanzania['weighted_bw'].sum())
print('overall mean birthweight from model baseline (tanzania) = ', + tanzania_bw_baseline['pop_x_bw'].sum()/100_000, )
print('overall mean sd of birthweight from model baseline (tanzania) = ', + tanzania_bwsd_baseline['pop_x_bw'].sum()/100_000)

lbwart_tanzania.head()

overall mean birthweight from artifact baseline (tanzania) =  3279.011285750329
overall mean birthweight from model baseline (tanzania) =  3274.713703128642
overall mean sd of birthweight from model baseline (tanzania) =  636.1296204147264


,location,sex,age_start,age_end,year_start,year_end,parameter,exposure,affected_entity,affected_measure,relative_risk,mean_bw,sex_ratio,weighted_bw
0,Tanzania,Female,0.0,0.019178,2017.0,2018.0,cat10,0.001377,all,excess_mortality_rate,968.702386,750.0,0.487808,0.503795
1,Tanzania,Female,0.0,0.019178,2017.0,2018.0,cat106,0.000258,all,excess_mortality_rate,171.751901,1250.0,0.487808,0.157271
2,Tanzania,Female,0.0,0.019178,2017.0,2018.0,cat11,0.000293,all,excess_mortality_rate,513.870580,750.0,0.487808,0.107128
3,Tanzania,Female,0.0,0.019178,2017.0,2018.0,cat116,0.000464,all,excess_mortality_rate,182.396469,1250.0,0.487808,0.283021
4,Tanzania,Female,0.0,0.019178,2017.0,2018.0,cat117,0.002365,all,excess_mortality_rate,61.431240,1750.0,0.487808,2.019143
